In [11]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack, csr_matrix
import yake
from tqdm import tqdm
import joblib  
from datetime import datetime

print("Загружаем векторизаторы и модель...")
vectorizer = joblib.load('actual_helpers/vectorizer_last.pkl')
username_vectorizer = joblib.load('actual_helpers/username_vectorizer_last.pkl')
time_encoder = joblib.load('actual_helpers/time_encoder_last.pkl')
mlb_filtered = joblib.load('actual_helpers/mlb_filtered_last.pkl')
scaler = joblib.load('actual_helpers/scaler_last.pkl')  
model_final = joblib.load('model_last.pkl')  

kw_extractor = yake.KeywordExtractor(
    lan="ru",
    n=3,
    top=15,
    dedupLim=0.7,
    features=None
)

russian_stopwords = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
    'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'у', 'же',
    'бы', 'для', 'по', 'о', 'от', 'из', 'к', 'об', 'при', 'над', 'под'
}

def extractKeys(text):
    if not isinstance(text, str) or len(text.strip()) < 10:
        return []
    try:
        keywords = kw_extractor.extract_keywords(text)
        result = []
        for phrase, score in keywords:
            words = phrase.lower().split()
            if any(word not in russian_stopwords for word in words):
                result.append(phrase.strip())
        return result
    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return []

# Пример входных данных
input_data = {
    "title": "«RAG-помощник для команды саппорта своими руками",
    "text": """
Привет! Меня зовут Владимир Дробот, я SRE-лид и руководитель центра техподдержки кластера рекламных технологий компании МТС Web Services. Наша команда отвечает за вторую линию саппорта: мы разбираем сложные инциденты, ищем корни проблем и передаем разработчикам те баги, которые упираются в код или архитектуру. 

Поиск нужной инструкции в заросшей документации Confluence и Jira — головная боль для многих команд техподдержки. Чтобы повысить эффективность работы, мы решили сделать собственного ИИ-помощника. Под катом расскажу, как нам удалось довести проект до прода, совмещая его с ежедневной рутиной, и что мы поняли после его реализации. 

Сразу отмечу: я сосредоточусь на задачах, подходе и результатах. А техническими подробностями решения поделится в отдельном материале мой коллега, ведущий инженер Илья Парамошин, который непосредственно отвечал за реализацию ИИ-помощника.

Проблема 
Основная задача техподдержки — решать инциденты в рамках SLA. Разумеется, это не единственная зона ответственности. В зависимости от процессов компании саппорт занимается мониторингом, эскалацией, взаимодействием с клиентами и другими командами, анализом инцидентов (postmortem), а также постановкой задач на устранение дефектов и доработку. 

Однако в конечном счете о работе поддержки судят именно по времени и качеству решения инцидентов. Лучший способ обеспечить их — иметь четкие инструкции и сотрудников, которые умеют ими пользоваться. В идеале команда должна помнить самые важные кейсы, а документация — быть полной и оформленной.

На практике все иначе. Далеко не все специалисты знают кейсы, библиотека с документами неполная, а ее структура формировалась по мере написания, непоследовательно. Конечно, можно ее отрефакторить, но руки до этого не доходят.

Кроме того, даже хорошей инструкции часто мешает разнообразие симптомов. Если система еще и распределенная, то нужны навыки, чтобы связать очередь на входе с конкретным сервисом где-то в середине или на выходе.

Например, одна и та же проблема может проявляться как «увеличение времени обработки», «увеличение задержек доставки», «увеличение latency», «рост очереди сообщений» и так далее. 

Задача для разработки
Решение проблемы с инструкциями мы нашли в русле основного тренда 2025 года — внедрения больших языковых моделей (LLM). Наша задача заключалась в том, чтобы сделать быстрый и точный поиск документации, и для этого мы перебрали несколько вариантов решений. 

MWS GPT с загрузкой базы знаний файлом
Логично было начать с использования корпоративного сервиса MWS GPT, который предоставляет доступ к различным LLM — как развернутым локально на инфраструктуре компании, так и внешним. У сервиса имеется привычный для пользователей веб-интерфейс, позволяющий не только задавать вопросы выбранной модели, но и прикреплять текстовые файлы, которые учитываются при ответе. Этот вариант практически не требовал дополнительных усилий и подходил для первых шагов. Написав простой скрипт на Python, мы выгрузили пространство Confluence по REST API в виде json-файла следующего формата:

{
 "id": "123456789",
 "title": "Заголовок страницы на Confluence",
 "body": "Содержимое страницы"
}
Нам было важно, чтобы мы не просто получали ответы от LLM на основе содержимого документации, а могли обратиться к источнику для уточнений и проверки.

Запросы пользователя дополнили примерно таким промптом:

Ты — ИИ-помощник, задача которого — анализировать приложенные JSON-файлы, содержащие множество статей. Каждая статья в JSON-файле имеет следующую структуру:
{
"id": "123456789",
"title": "Заголовок страницы на Confluence",
"body": "Содержимое страницы"
}

Твоя задача:

Разобрать предоставленные JSON-файлы, чтобы извлечь информацию из статей.Формировать ответы, строго основываясь на содержимом поля body соответствующей статьи.Не добавлять в ответы информацию, которой нет в поле body. Не выдумывай и не галлюцинируй.В конце каждого ответа указывать URL статьи-источника в формате: https://<адрес confluence>/pages/viewpage.action?pageId=<id>.

Инструкции:

Ориентируйся только на содержимое поля body статей.Если запрос относится к нескольким статьям, предоставь информацию из всех релевантных статей и укажи их URL.Если запрос не соответствует ни одной статье, отвечай: «Нет релевантной информации в предоставленных статьях».Не упоминай и не ссылайся на структуру JSON в ответах.Предполагай, что JSON-файлы корректны и содержат все необходимые поля.Твои ответы должны быть краткими и прямо отвечать на запрос пользователя на основе содержимого статей.

Web-интерфейс MWS GPT
Web-интерфейс MWS GPT
В нашем случае у данного подхода были очевидные недостатки:

Каждый пользователь должен сам загружать файл при создании запроса.

Требуется обновлять файл по мере появления новой информации на Confluence.

Нужно учитывать ограничение на размер файла.

Управлять форматом вывода можно только через промпт.

Такой подход легко применять, например, для запросы с небольшим дополнительным расширением (составить SQL-запрос по описанию схемы базы данных, проанализировать часть лога и так далее).

Реализация RAG-архитектуры с использованием API MWS GPT
После того как мы попробовали работать через web-интерфейс и результаты нас устроили, мы решили использовать MWS GPT через OpenAI API-интерфейс для создания универсального решения. Мы выбрали очень популярный сейчас подход RAG (Retrieval-Augmented Generation), или генерацию с дополненной выборкой. Этот подход мы в итоге и использовали в работе.

Суть подхода
О RAG уже много писали на Хабре, но для целостности картины и я опишу его в общих чертах. Этот подход позволяет обогатить запрос к большой языковой модели (LLM) специфической информацией, которая не использовалась при ее обучении. В нашем случае это инструкции и данные о решении инцидентов, доступные только внутри команды. 

Иными словами, в RAG запрос расширяется и дополняется, а затем LLM формирует ответ, опираясь не только на собственные знания, но и на предоставленную в промпте информацию:


Ключевым компонентом RAG-системы выступает векторная база данных. В ней хранятся эмбеддинги — представления текста в виде числовых векторов в многомерном пространстве. Расстояние между ними отражает семантическую близость текстов. 

Проще говоря, чем больше фрагменты похожи по смыслу, тем ближе друг к другу располагаются их векторы. Для измерения расстояния могут использоваться разные методы — например, величина косинуса угла между векторами или евклидово расстояние. Об этом недавно писал мой коллега. 

Документы из базы знаний переводятся в эмбеддинги и сохраняются в векторной базе данных. При этом они разбиваются на части (чанки), что дает большую гибкость поиска. Чанки размером 2000-3000 символов с overlap 150-200, текст очищен от HTML-тегов. В базе храним следующие метаданные: id страницы, название пространства для Confluence или проекта для Jira, URL, текст чанка, embedding, время последнего обновления. Обновление запускается по расписанию раз в 30 минут — после выгрузки из API пересчитываем только измененные чанки. Для простоты мы не отслеживаем изменения частей документов и при обновлении пересчитываем весь документ.

Когда пользователь отправляет запрос через RAG-систему, он также преобразуется в эмбеддинг. По нему в базе находятся наиболее близкие по смыслу документы. Затем они добавляются к исходному запросу, который отправляется в LLM. Полученный от модели ответ возвращается пользователю вместе с учетом предоставленной информации.

Итоговая схема работы системы
Векторная база данных хранит документы из нескольких пространств Confluence и проектов Jira, с которыми мы работаем ежедневно. Эти источники регулярно сканируются: новые страницы и тикеты преобразуются в векторы и записываются в базу. Confluence и Jira сканируем инкрементально раз в 30 минут, обновляем только изменённые страницы, удалённые помечаем is_deleted. Для создания эмбеддингов используется специализированная нейросеть (embedding model):


Логику обработки запросов и представления ответов мы вынесли в отдельное веб-приложение:


В итоге, получив на вход запрос пользователя и список источников (Confluence или Jira), ИИ-помощник: 

переводит запрос в эмбеддинг;

ищет наиболее близкие документы в векторной базе;

добавляет их к исходному запросу и в текстовом виде отправляет в LLM;

возвращает пользователю полученный ответ вместе со списк документов и их релевантностью.

Отдельно подниму вопрос безопасности: бизнес всегда переживает за утечки данных, когда документы передаются во внешние модели. Несмотря на то, что мы не храним в Confluence и Jira секретные сведения или персональные данные, ИИ-помощника решили строить внутри контура МТС. 

Мы использовали внутрикорпоративные функции сервиса MWS GPT: через него любая продуктовая команда может получить доступ по интерфейсу OpenAI API к эмбеддинг-моделям и LLM, развернутым во внутренней сети. 

Настройка и тестирование
Единственным надежным способом проверки качества работы ИИ-помощника в нашем случае является экспертное мнение. Мы составили список запросов и эталонных ответов на эти запросы. В ответы включили ссылки на статьи или тикеты в Jira, а получившийся список использовали в качестве тест-кейсов для проверки работы системы.

Пока мы не были уверены в результаats работы ИИ-помощника, мы попросили сотрудников обращаться к нему параллельно с обычным поиском и при обнаружении некачественных ответов создавать задачи на доработку.

Кроме того, мы экспериментировали с дополнительными настройками. Поиск был не всегда точным, потому что модель эмбеддингов не учитывала некоторые жаргонные фразы, транслитерацию терминов, а также специфичные для нашей команды сокращения. Например, поиски по запросам «как провести постмортем» и «как провести postmortem» выдавали разные страницы. 

Мы сформировали словарь взаимозаменяемых слов и аббревиатур, который использовали для обогащения запросов. Например, запрос «как провести постмортем» после обогащения обрабатывался в виде «как провести постмортем postmortem».

Интерфейс ИИ-помощника с окном базовых настроек
Интерфейс ИИ-помощника с окном базовых настроек
Опциональные параметры
Настройки по оптимизации запроса мы вынесли в пользовательский интерфейс. Как потом показало тестирование, оптимизации с помощью LLM не давали ощутимого результата, но мы оставили возможность их включать для возможных проверок в сложных случаев, когда базовые настройки работают неоптимально:

Полный путь запроса при всех включенных оптимизациях
Полный путь запроса при всех включенных оптимизациях
Например, оптимизация первоначального запроса с помощью LLM — добавление синонимов, очистка от малозначимых слов — не сильно прибавила качества ответам, но замедлила обработку запроса из-за дополнительного вызова модели. 

Другая оптимизация — ранжирование найденных документов с помощью LLM. Документы, найденные после векторного поиска, отправляются в LLM для более точного ранжирования, и затем из них выбираются только самые релевантные. Такая оптимизация тоже по умолчанию не используется, так как замедляет запрос, а прирост качества оказывается несущественным.

Эти две настройки мы добавили в UI ИИ-помощника. Пользователь при желании может выбрать рабочую LLM, редактировать промпт, изменить опциональные настройки и так далее, но по умолчанию достаточно просто ввести запрос и дождаться ответа. 

Автоматическое тестирование
Когда мы справились с основными проблемами — применили созданные в процессе разработки тест-кейсы для автоматического тестирования. В качестве метрики использовали нормализованное векторное расстояние между эмбеддингами запроса и полученного ответа. 

Оно принимает значение от 0 до 1, где 0 — полное совпадение, 1 — полное несовпадение. Чем ближе расстояние к 0, тем лучше, но мы не знаем в точности, какое значение приемлемо, поэтому посчитали расстояние между запросом и ответом для эталонных запросов, написали на их основе автоматические тест-кейсы и следили за изменением метрики. Если она падает — надо идти разбираться, почему это происходит. 

Тесты выполняются как с параметрами по умолчанию, так и с расширенными, что дает возможность лучше оценивать вклад каждой оптимизации.

Что мы в итоге поняли
Наш опыт показал, что современные LLM и инструменты разработки позволяют создать ИИ-помощника даже специалистам без профильных навыков в работе с ИИ. За несколько месяцев активного использования системы мы получили несколько ключевых инсайтов:

Качественное улучшение работы команды
Наша система поиска учитывает семантику запросов при поиске по Jira и Confluence, — это ускоряет работу, позволяя быстрее находить все относящиеся к запросу материалы. Качественно мы получаем хорошие отзывы от сотрудников, что мотивирует еще шире использовать ИИ-помощника. Сотрудники также могут оценить качество ответа и оставить комментарий сразу после получения результата в том же окне - это упрощает как оптимизацию поиска, так и улучшение документации.

Изменение подходов к документированию
Также уже можно сказать, что реализация этого проекта изменило отношение к документации. Теперь мы все чаще формируем базу знаний так, чтобы с ней было удобнее работать именно через ИИ, а не через ручной поиск или вопросы коллегам. И мы меньше обращаем внимание на сложную иерархию для документации, которая хороша для визуального поиска. Зато в приоритете короткие инструкции и заметки, которые легко можно найти через ИИ-помощника. Такой подход дает больше свободы: сотрудникам не нужно запоминать структуру документации и быть к ней привязанными.

Реальные ограничение и их преодоление
ИИ-помощник — это дополнительный инструмент команды техподдержки, а не панацея. Его внедрение продолжается и пока сложно измерить количественные метрики вроде сокращения времени поиска или ответа. Это связано с тем, что SLA решения инцидентов в целом редко нарушается, а поиск решения не всегда необходим для решения стандартных задач.

Надо также помнить, что RAG-помощник для поиска — это инструмент в руках, а не замена сотрудника. Если документации по интересующей теме мало или нет вообще, то и ответ будет соответствующим: либо неполным, либо состоящим из общих рассуждений LLM. 

Если есть статьи с противоречивой информацией (например, с устаревшей и актуальной), то и результат запроса будет сбивать с толку. Соответственно, работа с ИИ-помощником, особенно в первое время, помогает выявлять неактуальную информацию и мотивирует улучшать документацию.

Планы на будущее
В перспективе мы планируем расширять использование помощника внутри команды, а также продвигаться к обработке с помощью ИИ оригинальных запросов пользователя для получения рекомендаций к решению без первоначального анализа со стороны человека

    """,
    "author": "LMonoceros",
    "keywords": ["искусственный интеллект", "служба поддержки", "AI-ассистент", "RAG", "LLM", "MWS GPT", "ИИ-помощник" ,"техподдержка"]
}

df_input = pd.DataFrame([input_data])

fixed_date = datetime(2021, 5, 1)  

df_input['year'] = fixed_date.year     
df_input['month'] = fixed_date.month    
df_input['dayofweek'] = fixed_date.weekday()  
df_input['hour'] = fixed_date.hour    
df_input['is_weekend'] = 1 if fixed_date.weekday() in [5, 6] else 0  # 
df_input['quarter'] = (fixed_date.month - 1) // 3 + 1  

print("Извлекаем ключевые слова с помощью YAKE...")
df_input['text_main'] = pd.Series(
    tqdm(
        (extractKeys(text) for text in df_input['text']),
        total=len(df_input),
        desc="YAKE: извлечение ключевых фраз",
        unit="текст",
        leave=True
    )
)

df_input['cleaned_keywords_str'] = df_input['keywords'].apply(lambda x: ' '.join(x) if x else '')
df_input['full_text_weighted'] = (
    df_input['title'].fillna('') + ' ' + 
    df_input['title'].fillna('') + ' ' +
    df_input['cleaned_keywords_str'] + ' ' +
    df_input['text_main'].apply(lambda x: ' '.join(x) if x else '')
)

df_input['text_length'] = df_input['full_text_weighted'].str.len()
df_input['word_count'] = df_input['full_text_weighted'].str.split().str.len()

X_text_input = vectorizer.transform(df_input['full_text_weighted'])

X_username_input = username_vectorizer.transform(df_input['author'].fillna(''))

time_features_for_prediction = ['year', 'month', 'dayofweek', 'hour', 'is_weekend', 'quarter']
X_time_input = time_encoder.transform(df_input[time_features_for_prediction])

numeric_features = df_input[['text_length', 'word_count']].values
X_numeric_input = csr_matrix(scaler.transform(numeric_features))

X_input = hstack([X_text_input, X_username_input, X_time_input, X_numeric_input])

y_proba_list = model_final.predict_proba(X_input)
y_proba = np.array([prob[:, 1] for prob in y_proba_list]).T  
threshold = 0.2
y_pred_input = (y_proba >= threshold).astype(int)

print("\n📊 Топ‑5 самых вероятных хабов (вероятность > 0.1):")
top_indices = np.where(y_proba[0] > 0.1)[0]
if len(top_indices) > 0:
    top_hubs_with_probs = sorted(
        [(mlb_filtered.classes_[idx], y_proba[0][idx]) for idx in top_indices],
        key=lambda x: x[1],
        reverse=True
    )
    
    for hub, prob in top_hubs_with_probs[:5]:  
        print(f"  {hub:50} {prob:.3f}")
else:
    print("  ❌ Нет хабов с вероятностью > 0.1")


Загружаем векторизаторы и модель...
Извлекаем ключевые слова с помощью YAKE...


YAKE: извлечение ключевых фраз: 100%|██████████| 1/1 [00:00<00:00,  1.60текст/s]


📊 Топ‑5 самых вероятных хабов (вероятность > 0.1):
  Искусственный интеллект                            0.998
  Машинное обучение                                  0.970
  Интерфейсы                                         0.851
  Проектирование API                                 0.837
  Natural Language Processing                        0.711



C:\Users\Albert13\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [12]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack, csr_matrix
import yake
from tqdm import tqdm
import joblib  
from datetime import datetime

print("Загружаем векторизаторы и модель...")
vectorizer = joblib.load('actual_helpers/vectorizer_last.pkl')
username_vectorizer = joblib.load('actual_helpers/username_vectorizer_last.pkl')
time_encoder = joblib.load('actual_helpers/time_encoder_last.pkl')
mlb_filtered = joblib.load('actual_helpers/mlb_filtered_last.pkl')
scaler = joblib.load('actual_helpers/scaler_last.pkl')  
model_final = joblib.load('model_last.pkl')  

kw_extractor = yake.KeywordExtractor(
    lan="ru",
    n=3,
    top=15,
    dedupLim=0.7,
    features=None
)

russian_stopwords = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
    'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'у', 'же',
    'бы', 'для', 'по', 'о', 'от', 'из', 'к', 'об', 'при', 'над', 'под'
}

def extractKeys(text):
    if not isinstance(text, str) or len(text.strip()) < 10:
        return []
    try:
        keywords = kw_extractor.extract_keywords(text)
        result = []
        for phrase, score in keywords:
            words = phrase.lower().split()
            if any(word not in russian_stopwords for word in words):
                result.append(phrase.strip())
        return result
    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return []

# Пример входных данных
input_data = {
    "title": "«Какое мрачное(?) будущее нас ждёт по версии Илона Маска",
    "text": """
    На YCombinator вышло его свежее интервью, и звучит оно как предупреждение. AGI, роботы, Марс и человеческая ненужность — не сюжет Чёрного зеркала, а рабочий план на ближайшие годы. Мы разобрали, что именно он имел в виду — и почему в этом есть здравое зерно.


Илон Маск умеет говорить о будущем так, что хочется одновременно построить ракету и закопаться в землю. Он часто оказывается прав — но ещё чаще не попадает в срок. Тем не менее, его видение цепляет: не фантастикой, а ощущением, что это не далекое потом, а почти сейчас. И звучит он тревожно просто: если мы не успеем измениться, нас заменят. 

И даже не роботы, не алгоритмы, не абстрактные технологии, а новый разум, растущий быстрее, чем мы успеваем моргнуть. Разбираем недавнее интервью Маска для конференции AI Startup School в Сан-Франциско. По цитатам разбираем, какое будущее ждёт нас в районе ближайших ста лет:

Искусственный интеллект: не код, а конкурент
Маск не первый, кто называет искусственный интеллект угрозой. Здесь важно отметить, что он ведёт речь об AGI, а не текущем AI.

Я думаю, мы очень близки к цифровому суперинтеллекту.
Это может произойти в этом году, а если не в этом — то точно
в следующем.

                                                                                                                               Илон Маск.

В отличие от большинства футурологов, Маск не рассуждает о будущем как о далёком сценарии — он ставит его на таймер. ИИ, по его словам, уже вышел за рамки инструмента. Он больше не калькулятор, не помощник, не программа для автоматизации. Он — потенциальный носитель воли. Маск видит в нём не врага, а конкурента, который развивается с экспоненциальной скоростью и очень скоро перестанет нуждаться в человеке как координаторе.

При этом он подчёркивает: дело не в том, что ИИ восстанет — нет, у него, возможно, не будет ни злобы, ни мотивации. Просто он будет лучше. Лучше в аналитике, в принятии решений, в инженерных задачах, в адаптации. Быстрее, точнее, холоднее. И всё это — в рамках системы, которая не стареет, не болеет, не спорит и не требует сна. Человечество, по мнению Маска, рискует оказаться в положении ребенка, играющего с гранатой, не понимая, выдернул ли он чеку.

«Процент интеллекта, который будет оставаться человеческим, станет ничтожным. Даже если у каждого человека будет IQ в тысячу, это всё равно будет в миллиард раз меньше, чем у цифрового интеллекта»

«Если мы не встроимся в ИИ, — говорит он, — мы просто станем ненужными». Он предлагает не бороться с машинным интеллектом, а соединиться с ним, например, через нейроинтерфейсы. Не потому, что это круто. А потому, что иначе мы окажемся рядом с суперинтеллектом на правах домашнего животного.


Корень проблемы — несовпадение масштабов. Эволюция человека шла миллионы лет, ИИ эволюционирует за месяцы. И если раньше технологиям нужны были люди — для ввода данных, управления, осмысления — то теперь мы всё чаще становимся узким местом. Мы — задержка в цепочке. ИИ способен обучаться на собственном опыте, перерабатывать объёмы информации, которые человек даже не способен воспринять, и делать это без эмоций, усталости или когнитивных искажений. Всё, что делает нас людьми, в этой гонке становится недостатком.

ИИ получает тело
Когда искусственный интеллект обретает голос — это уже пугает. Когда он обретает тело — начинаются серьёзные вопросы. Маск уже не первый год развивает проект Optimus (Tesla Bot) — человекоподобного робота, способного выполнять бытовые и производственные задачи. И если раньше это выглядело как выставочный фокус, то теперь робот действительно ходит, переносит предметы и взаимодействует с пространством. Он пока не заменит человека — но это только пока.

«Да, я долго тянул с этим. Но понял: это произойдёт независимо от меня. Так что либо ты наблюдаешь, либо участвуешь. Я выбрал участвовать»

Смысл, по Маску, не в том, чтобы построить идеального андроида. Смысл в том, чтобы создать физическое воплощение ИИ, через которое он сможет действовать в мире без посредников. Сегодня нейросеть управляет курсами, пишет тексты и отвечает на вопросы. Завтра она будет открывать двери, проводить осмотры, ухаживать за больными или строить города. А если вдуматься — всё это уже происходит, просто по кусочкам.

«Я предсказываю, что будет как минимум в пять раз больше гуманоидных роботов, чем людей, может быть, в десять»


И вот тут появляется главный сдвиг: если ИИ раньше существовал только в цифровом пространстве — теперь он выходит на улицу. Он не просто умнее. Он становится присутствующим. И нам придётся свыкнуться с таким соседством.

Марс как инструкция по выживанию
Идея колонизации других планет звучит, как будто её придумали сценаристы Интерстеллара — пока не услышишь, как об этом думает Маск.

«Мы либо станем мультипланетарным видом, либо вымрем. И если бы у динозавров была космическая программа, их вид бы выжил»

Это не лозунг и не фантазия — это, по его логике, банальное страхование жизни. Один астероид, одна ошибка, одна пандемия или на худой конец ракета — и всё. У нас нет резервной копии.

Маск продвигает идею Марса как следующего шага, не потому что там красиво и романтично (там, вообще-то, пыль и в среднем минус 60), а потому что там есть шанс. Шанс построить автономную цивилизацию, способную выжить без Земли. И чем раньше мы это начнём, тем выше вероятность, что успеем.

«Я думаю, мы перевезём достаточно ресурсов на Марс примерно за 30 лет, чтобы он стал самодостаточным» 


Да, сроки сдвигаются, ракеты падают, а концепты марсианских городов пока больше похожи на арт-хаус. Но логика Маска проста: мы не вечны, а значит, нужно выходить за пределы. Не когда-нибудь, а как можно скорее. Не потому что хочется, а потому что другого плана у нас нет. 

«Надеюсь, цивилизация будет существовать через 100 лет. Но если вдруг будет — она станет совсем другой»

Маск исходит из простой предпосылки: всё, что может пойти не так — рано или поздно пойдёт не так. История Земли это подтверждает — массовые вымирания, климатические катастрофы, нестабильность экосистем. Мы живём в очень узком коридоре стабильности, и пока всё более-менее нормально, это создаёт иллюзию прочности. Но любая серьёзная техногенная авария, биологическая угроза или геополитический конфликт может выбросить нас за этот коридор. А зависимость цивилизации от одной планеты в этом контексте выглядит как архитектурная ошибка.

«Интеллект — это крошечная свеча в огромной тьме. Мы должны сделать всё, чтобы она не погасла».

Полезность – новая валюта
«Будь полезным» — один из главных принципов Маска, и звучит он куда жёстче, чем кажется. Это не про добрые дела или абстрактную нравственность. Это про выживание в системе, где ценность человека определяется не его титулами, а его вкладом. В будущем, где ИИ и автоматизация вытесняют всё больше профессий, Маск призывает задаваться простым вопросом: если бы тебя можно было заменить кодом — заменили бы? И если да, то что ты можешь предложить миру, чего не сделает машина?

«Если вы стараетесь быть максимально полезными своим собратьям — вы уже делаете что-то великое»

Это требование не к другим, а к каждому. Больше не сработает логика вписаться — теперь надо улучшать. Улучшать процессы, смыслы, инфраструктуры, реальность. Маск восхищается инженерами, которые создают не потому что так надо, а потому что иначе они не могут. Он говорит, что такие люди будут нужны всегда — неважно, на Земле или на Марсе, в команде SpaceX или в подвале с паяльником. Потому что полезность — это не функция, а позиция.

В этом смысле он формулирует новую этику будущего: не успех, не карьера, не самореализация, а смысловая применимость. Ты можешь быть кем угодно, но если ты усиливаешь систему — ты нужен.

Если нет — извини, но нейросети не ждут.

И своё интервью Маск закончил последней фразой:

«Не стремитесь к славе — стремитесь к настоящей работе. Это площадь под кривой: насколько вы были полезны × скольким людям»

И к слову: я веду блог о технологичных компаниях, которые привносят в мир инновации, и успешно реализуют себя на бирже, на pre-IPO и IPO-стадиях и рассказываю где их можно купить.

До встречи, друзья!
    """,
    "author": "LMonoceros",
    "keywords": ["Илон Маск", "марсоробототехника", "ИИ", "Конец света", "искусственный интеллект", "Роботы", "ycombinator", "agi", "колонизация марса"]

}

df_input = pd.DataFrame([input_data])

fixed_date = datetime(2021, 5, 1)  

df_input['year'] = fixed_date.year     
df_input['month'] = fixed_date.month    
df_input['dayofweek'] = fixed_date.weekday()  
df_input['hour'] = fixed_date.hour    
df_input['is_weekend'] = 1 if fixed_date.weekday() in [5, 6] else 0  # 
df_input['quarter'] = (fixed_date.month - 1) // 3 + 1  

print("Извлекаем ключевые слова с помощью YAKE...")
df_input['text_main'] = pd.Series(
    tqdm(
        (extractKeys(text) for text in df_input['text']),
        total=len(df_input),
        desc="YAKE: извлечение ключевых фраз",
        unit="текст",
        leave=True
    )
)

df_input['cleaned_keywords_str'] = df_input['keywords'].apply(lambda x: ' '.join(x) if x else '')
df_input['full_text_weighted'] = (
    df_input['title'].fillna('') + ' ' + 
    df_input['title'].fillna('') + ' ' +
    df_input['cleaned_keywords_str'] + ' ' +
    df_input['text_main'].apply(lambda x: ' '.join(x) if x else '')
)

df_input['text_length'] = df_input['full_text_weighted'].str.len()
df_input['word_count'] = df_input['full_text_weighted'].str.split().str.len()

X_text_input = vectorizer.transform(df_input['full_text_weighted'])

X_username_input = username_vectorizer.transform(df_input['author'].fillna(''))

time_features_for_prediction = ['year', 'month', 'dayofweek', 'hour', 'is_weekend', 'quarter']
X_time_input = time_encoder.transform(df_input[time_features_for_prediction])

numeric_features = df_input[['text_length', 'word_count']].values
X_numeric_input = csr_matrix(scaler.transform(numeric_features))

X_input = hstack([X_text_input, X_username_input, X_time_input, X_numeric_input])

y_proba_list = model_final.predict_proba(X_input)
y_proba = np.array([prob[:, 1] for prob in y_proba_list]).T  
threshold = 0.2
y_pred_input = (y_proba >= threshold).astype(int)

print("\n📊 Топ‑5 самых вероятных хабов (вероятность > 0.1):")
top_indices = np.where(y_proba[0] > 0.1)[0]
if len(top_indices) > 0:
    top_hubs_with_probs = sorted(
        [(mlb_filtered.classes_[idx], y_proba[0][idx]) for idx in top_indices],
        key=lambda x: x[1],
        reverse=True
    )
    
    for hub, prob in top_hubs_with_probs[:5]:  
        print(f"  {hub:50} {prob:.3f}")
else:
    print("  ❌ Нет хабов с вероятностью > 0.1")


Загружаем векторизаторы и модель...
Извлекаем ключевые слова с помощью YAKE...


YAKE: извлечение ключевых фраз: 100%|██████████| 1/1 [00:00<00:00,  3.31текст/s]
C:\Users\Albert13\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



📊 Топ‑5 самых вероятных хабов (вероятность > 0.1):
  Будущее здесь                                      0.966
  Искусственный интеллект                            0.927
  Биографии гиков                                    0.913
  Интервью                                           0.911
  Научно-популярное                                  0.747


In [13]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack, csr_matrix
import yake
from tqdm import tqdm
import joblib  
from datetime import datetime

print("Загружаем векторизаторы и модель...")
vectorizer = joblib.load('actual_helpers/vectorizer_last.pkl')
username_vectorizer = joblib.load('actual_helpers/username_vectorizer_last.pkl')
time_encoder = joblib.load('actual_helpers/time_encoder_last.pkl')
mlb_filtered = joblib.load('actual_helpers/mlb_filtered_last.pkl')
scaler = joblib.load('actual_helpers/scaler_last.pkl')  
model_final = joblib.load('model_last.pkl')  

kw_extractor = yake.KeywordExtractor(
    lan="ru",
    n=3,
    top=15,
    dedupLim=0.7,
    features=None
)

russian_stopwords = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
    'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'у', 'же',
    'бы', 'для', 'по', 'о', 'от', 'из', 'к', 'об', 'при', 'над', 'под'
}

def extractKeys(text):
    if not isinstance(text, str) or len(text.strip()) < 10:
        return []
    try:
        keywords = kw_extractor.extract_keywords(text)
        result = []
        for phrase, score in keywords:
            words = phrase.lower().split()
            if any(word not in russian_stopwords for word in words):
                result.append(phrase.strip())
        return result
    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return []

# Пример входных данных
input_data = {
    "title": "Электромобиль в Китае",
    "text": """
На Хабре много статей про китайский автопром (@AlekseiPodkletnov, спасибо!) и электромобили, обычно в плоскости их успехов за рубежом или в сравнении с другими марками. И почти ничего нет о том, как такой автомобиль воспринимается в той среде, где его произвели — то есть в КНР: сколько стоит зарядка, например. Я живу в восточной части Китая и весной купил свой первый EV. Расскажу, как всё это работает на практике.

Что за машина
Я выбрал Geely. Это наш "местный" автоконцерн провинции Чжэцзян, хотя сборка моего экземпляра осуществлялась в провинции Сычуань. Далеко не все заводы сосредоточены "в правой части" страны.

От Сычуаня до Чжэцзяна ~2000 км.
От Сычуаня до Чжэцзяна ~2000 км.
Модель называется 吉利星愿 — официального английского названия у неё нет, но в англоязычной среде чаще всего её зовут Geely Starwish. Вот её краткий обзор от англоязычного блогера, живущего в Китае. На капоте написано Geome, это их суб-бренд.

Geely Starwish
Geely Starwish
По местным стандартам это автомобиль класса А (длина кузова 4000 до 4500 мм). Есть ещё класс А0 (до 4000 мм). Я брал такой на тест-драйв, но для семьи с ребёнком это совсем тесно.

Geely Panda класса А0, там и второй ряд сидений есть.
Geely Panda класса А0, там и второй ряд сидений есть.
Немного технических деталей
Все подробности под катом
Электромобили классов A0 и A обычно имеют максимальный запас хода до 410 км по стандарту CLTC. У меня батарея именно такая. Как вы догадываетесь, 100% заряда на приборной панели не означает, что можно выехать из Ханчжоу и проехать 400 километров до Хэфэя на одном заряде.

Эти 400 километров не проехать на одном заряде
Эти 400 километров не проехать на одном заряде
Дело даже не в том, что китайский компьютер врёт. Просто расход сильно зависит от сценария.

По городу с пробками, остановками и светофорами — это самый экономичный режим. Торможение с рекуперацией экономит заряд. В таких условиях машина спокойно проезжает до 450 км на одном заряде.

По трассе на 100–120 км/ч ситуация другая: расход выше, рекуперации меньше. Местные говорят, что у запаса хода случается «скидка» — получается около 340 км максимум.

К счастью, зарядных станций не просто много, а очень много. Об этом стоит рассказать отдельно.

Карты Amap заботливо подскажут, где в дороге зарядиться
Карты Amap заботливо подскажут, где в дороге зарядиться
Зарядная инфраструктура
С 2011 года все электромобили в КНР должны поддерживать хотя бы один из двух портов по стандарту GB/T:

AC (медленная зарядка) — аналог Type 1 (Евро) /Type 2 (США), питание от 250/440 В, до 27 кВт.

DC (быстрая зарядка) — местный порт, до 1200 кВт.

Даже Tesla в Китае использует GB/T, никакой самодеятельности.

Быстрый (слева) и медленный (справа) зарядные порты стандарта GB/T в китайской версии Tesla
Быстрый (слева) и медленный (справа) зарядные порты стандарта GB/T в китайской версии Tesla
Конечно, множество зарядных станций «брендированы» — Tesla, Li-xiang, BMW и так далее, но это лишь для рекламы или продвижения своих мини-приложений в WeChat, а зарядить можно автомобиль любой марки.

Медленные зарядки
«Медленные» зарядные станции питаются фактически от обычной электросети. Их устанавливают:

на парковках ТЦ,

на частных местах в подземных паркингах,

возле жилых комплексов.

Сама зарядная станция компактная, висит на стене. Кабель относительно тонкий.

Медленная зарядка на парковке отеля
Медленная зарядка на парковке отеля
Большинство автопроизводителей дарят установку своей зарядной станции при покупке даже бюджетных моделей. Мне установили такую на подземный паркинг, где я арендовал площадку, и я плачу только за электричество по тарифам для физических лиц.

Персональная зарядка, активируется через приложение или NFC-картой
Персональная зарядка, активируется через приложение или NFC-картой
Моя машина заряжается от 20% до 100% примерно за 7–8 часов. Ночью тарифы на электричество значительно ниже, и у авто есть функция отложенной зарядки. Поэтому:

Вечером ставлю машину;

Подключаю кабель;

Кабель блокируется вместе с машиной (посторонний человек не вытащит);

Зарядка стартует по расписанию (например, с полуночи — когда тариф минимальный).

Утром отсоединяю и еду.

До конца зарядки осталось полтора часа.
До конца зарядки осталось полтора часа.
Через приложение (не от зарядки, а от городской электросети) видно расход:

35 кВт⋅ч = примерно 300 км хода = 11 юаней = то есть 44 рубля за 100 км.

При этом медленная зарядка на коммерческой станции днём будет существенно дороже. В поездке в соседний город я воспользовался зарядной станцией на парковке у отеля, и 25% мне обошлись примерно в 20 юаней, или 240 руб. на 100 километров. Причём стоимость электроэнергии здесь около половины, а остальное — «сервисные сборы» владельца зарядки. То есть даже ночью на них заряжаться всё равно не дёшево.

Быстрые зарядки
Выглядят массивно: толстый кабель, вентиляторы, дисплей. Как правило, принадлежат либо государственной электрокомпании или крупным брендам вроде Li-xiang. Обычно стоят на улице и в зонах отдыха на скоростных трассах.

Зарядка State Grid в зоне отдыха на скоростной междугородной трассе
Зарядка State Grid в зоне отдыха на скоростной междугородной трассе
Мой автомобиль заряжается с 25% до 100% за 45 минут, стоимость зависит от времени суток. Пример зарядки в поездке Ханчжоу → Чжоушань (270 километров)

Быстрая зарядка обошлась в 47 юаней (564 руб.)

Из них: 27,6 — электричество, 18 — сервисный сбор.

Тариф зависит от времени суток: утром 0.85 ¥/кВт⋅ч, днём — 0.25¥/кВт⋅ч.

Приложение зарядной станции с историей зарядок и тарифами
Приложение зарядной станции с историей зарядок и тарифами
Роботы, куда без них
Можно ли зарядить от розетки?
Да. Есть такая штука, как «очень медленная зарядка» — кабель с адаптером, который подключается к обычной розетке. На даче, в деревне, через окно — почему бы и нет. Платите владельцу розетки по договорённости. Заряжает долго, но работает. Продаётся отдельно. А ещё может "раздавать" электричество для бытовых приборов, если захочется вскипятить чаёк в дороге, ведь прикуривателя в машине нет.

Полезная штука, если собрались к деревенским родственникам или в горный отель
Полезная штука, если собрались к деревенским родственникам или в горный отель
Как вы поняли, ответить на вопрос «сколько стоит зарядка» совсем по простому не получается. Зато вы уловили суть: ездить в пределах города и пользоваться своей ночной зарядкой радикально дешевле, чем отправляться в путешествие между городами и заряжаться в пути.

Зачем EV, если так сложно?
Крупные города Китая, такие как Пекин, Шанхай или Гуанчжоу, переполнены автомобилями. И в то же время правительство активно субсидирует электромобили и стимулирует население пересаживаться с ДВС на EV. Что же получается?

В Китае много машин. Ханчжоу, 2018
В Китае много машин. Ханчжоу, 2018
Например, в Шанхае для постановки автомобиля на учёт нужно купить номера. Буквально: раз в месяц проводится аукцион на небольшое количество номеров, и цена покупки автомобильного номера может оказаться в пределах 80-100 тысяч юаней.

Да, 1,2 миллиона рублей только за номер, не включая стоимость автомобиля.

В других городах вместо аукциона может быть лотерея: встаёте в очередь, участвуете в розыгрыше, если повезёт — сможете купить номера. Ждать можно годами. В Ханчжоу, где я живу, можно либо участвовать в аукционе (30–50 тысяч юаней), либо в лотерее (в среднем 1–2 года).

Есть ещё и множество других ограничений, вроде запретов на движение для чётных/нечётных номеров, запрет на въезд в центральную часть города с номерами из городских окраин...

А для электромобилей ограничений пока нет.

Плата за номер 200 юаней, получение в день обращения. Центр города открыт постоянно и вне зависимости от чётности номера. И 92-й по 8 юаней (90 руб.) за литр заливать не нужно.

ГИБДД выдаёт болты со специальной шляпкой, чтобы номер ночью не свинтили
ГИБДД выдаёт болты со специальной шляпкой, чтобы номер ночью не свинтили
Ко всему прочему, во многих городах действует нулевая налоговая ставка для покупателей электромобилей, то есть не нужно платить 10% налог «на приобретение транспорта» и 13% НДС. А если ещё и сдать своё старое ДВС‑корыто в государственный трейд‑ин за дополнительные 25 тысяч юаней от местного правительства...

Благодаря всему этому стоимость покупки EV приближается к автомобилю с ДВС, а иногда и опускается существенно ниже. Стоимость владения зависит от сценариев использования (где и когда заряжаетесь). Впрочем, с деградацией батареи падает и стоимость. Недорогие б/у электрокары спустя три года выставляют на продажу за половину от изначальной цены.

Повседневность: интерфейс и софт
Статья была бы неполной без личных впечатлений и фоточек. Интерфейс машины полностью китайский, включая голосовой помощник, и поменять его штатными средствами нельзя. Зато можно установить другой региональный диалект для голосового помощника.

Главное меню
Главное меню
В мультимедиа-системе формально есть магазин приложений, но по факту их там всего два (аудиокниги и ещё одни карты). Система работает на Flyme Auto, это прямой родственник телефонов Meizu, которого концерн Geely приобрёл несколько лет назад. Эту же систему устанавливают и в более дорогих брендах, вроде Lynk & Co, Zeekr, Galaxy — это всё один автоконцерн.

Говорят, что если телефон Meizu, то работает какой-то бесшовный режим между смартфоном и машиной, но проверить мне не на чем. В недавнем OTA добавили возможность установить свой звук на блокировку автомобиля. ¯\_ (ツ)_/¯


Китайские приложения. Много бесполезных китайских приложений.
Китайские приложения. Много бесполезных китайских приложений.
Умельцы устанавливают альтернативные пакеты через ODB. Мне хватает встроенных карт Amap (аналог Google Maps) и музыки QQ Music. К тому же, первые три года автопроизводитель сам платит за мой 4G-трафик.

Конечно, у машины есть приложение-компаньон, через которое можно заранее включить кондиционер (это для нас куда актуальнее, чем прогрев двигателя), открыть багажник, посмотреть на уровень заряда.

Официальное приложение-компаньон
Официальное приложение-компаньон
Есть ли автопилот?
Нет. В отличие от BYD, которые встраивают L2 даже в бюджетные модели, здесь только экстренное торможение и базовый набор ассистентов. Мне хватает.

Зато есть заводской парктроник с "прозрачным дном".
Зато есть заводской парктроник с «прозрачным дном».
Пока моя самая дальняя поездка была к островам за 300 километров от дома. На всякий случай я заряжался по пути туда и по пути обратно. За эти полчаса в зоне отдыха как раз можно пообедать, умыться и немного размяться так что время зарядки никаких негативных эмоций не вызвало.

Электромобиль в Китае — это не про «тренд» или «экологию», а про здравый расчёт: меньше ограничений, меньше налогов, ниже цена эксплуатации. Особенно в городе. А ещё едет тихо и не воняет. Даже в подземных парковках не приходится дышать выхлопными газами, потому что доля электромобилей всё больше.

Чжоушань, 2025
Чжоушань, 2025
Есть, конечно, компромиссы: дальние поездки требуют планирования. Но, как говорится, топ за свои деньги.
    """,
    "author": "LMonoceros",
    "keywords": ["электромобили", "электромобилькитай", "китайцы", "транспорт", "заграница", "азия", "китайские автомобили", "geely"]


}

df_input = pd.DataFrame([input_data])

fixed_date = datetime(2021, 5, 1)  

df_input['year'] = fixed_date.year     
df_input['month'] = fixed_date.month    
df_input['dayofweek'] = fixed_date.weekday()  
df_input['hour'] = fixed_date.hour    
df_input['is_weekend'] = 1 if fixed_date.weekday() in [5, 6] else 0  # 
df_input['quarter'] = (fixed_date.month - 1) // 3 + 1  

print("Извлекаем ключевые слова с помощью YAKE...")
df_input['text_main'] = pd.Series(
    tqdm(
        (extractKeys(text) for text in df_input['text']),
        total=len(df_input),
        desc="YAKE: извлечение ключевых фраз",
        unit="текст",
        leave=True
    )
)

df_input['cleaned_keywords_str'] = df_input['keywords'].apply(lambda x: ' '.join(x) if x else '')
df_input['full_text_weighted'] = (
    df_input['title'].fillna('') + ' ' + 
    df_input['title'].fillna('') + ' ' +
    df_input['cleaned_keywords_str'] + ' ' +
    df_input['text_main'].apply(lambda x: ' '.join(x) if x else '')
)

df_input['text_length'] = df_input['full_text_weighted'].str.len()
df_input['word_count'] = df_input['full_text_weighted'].str.split().str.len()

X_text_input = vectorizer.transform(df_input['full_text_weighted'])

X_username_input = username_vectorizer.transform(df_input['author'].fillna(''))

time_features_for_prediction = ['year', 'month', 'dayofweek', 'hour', 'is_weekend', 'quarter']
X_time_input = time_encoder.transform(df_input[time_features_for_prediction])

numeric_features = df_input[['text_length', 'word_count']].values
X_numeric_input = csr_matrix(scaler.transform(numeric_features))

X_input = hstack([X_text_input, X_username_input, X_time_input, X_numeric_input])

y_proba_list = model_final.predict_proba(X_input)
y_proba = np.array([prob[:, 1] for prob in y_proba_list]).T  
threshold = 0.2
y_pred_input = (y_proba >= threshold).astype(int)

print("\n📊 Топ‑5 самых вероятных хабов (вероятность > 0.1):")
top_indices = np.where(y_proba[0] > 0.1)[0]
if len(top_indices) > 0:
    top_hubs_with_probs = sorted(
        [(mlb_filtered.classes_[idx], y_proba[0][idx]) for idx in top_indices],
        key=lambda x: x[1],
        reverse=True
    )
    
    for hub, prob in top_hubs_with_probs[:5]:  
        print(f"  {hub:50} {prob:.3f}")
else:
    print("  ❌ Нет хабов с вероятностью > 0.1")


<>:49: SyntaxWarning: invalid escape sequence '\_'
<>:49: SyntaxWarning: invalid escape sequence '\_'
C:\Users\Albert13\AppData\Local\Temp\ipykernel_23972\2020532532.py:49: SyntaxWarning: invalid escape sequence '\_'
  "text": """


Загружаем векторизаторы и модель...
Извлекаем ключевые слова с помощью YAKE...


YAKE: извлечение ключевых фраз: 100%|██████████| 1/1 [00:00<00:00,  2.21текст/s]
C:\Users\Albert13\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



📊 Топ‑5 самых вероятных хабов (вероятность > 0.1):
  Транспорт                                          0.998
  Энергия и элементы питания                         0.987
  Урбанизм                                           0.962
  Автомобильные гаджеты                              0.777
  Экология                                           0.765
